In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
def reset_tf():
    global sess
    sess.close()
    tf.reset_default_graph()
    sess = tf.InteractiveSession()

In [4]:
sess = tf.InteractiveSession()

In [46]:
reset_tf()

n_h = 5
n_steps = 4
n_x = 3

X_raw = tf.placeholder(tf.int32, [None, n_steps])
Y_raw = tf.placeholder(tf.int32, [None, n_steps])
H_init = tf.placeholder(tf.float32, [None, n_h])

W_xh = tf.get_variable("W_xh", shape=[n_x, n_h])
W_hh = tf.get_variable("W_hh", shape=[n_h, n_h])
W_ha = tf.get_variable("W_ha", shape=[n_h, n_x])
b_h = tf.get_variable("b_h", shape=[1, n_h])
b_a = tf.get_variable("b_a", shape=[1, n_x])

X   = [None] + tf.unstack(tf.one_hot(X_raw, n_x), axis=1)
Y   = [None] + tf.unstack(tf.one_hot(Y_raw, n_x), axis=1)
H   = [H_init] + [None] * n_steps
Z_h = [None] * (n_steps+1)
Z_a = [None] * (n_steps+1)
A   = [None] * (n_steps+1)

for i in range(1, n_steps):
    Z_h[i] = tf.matmul(X[i], W_xh) + tf.matmul(H[i - 1], W_hh) + b_h
    H[i]   = tf.tanh(Z_h[i])
    Z_a[i] = tf.matmul(H[i], W_ha) + b_a
    A[i]   = tf.nn.softmax(Z_a[i])
    
loss = tf.constant(0.0)
for i in range(1, n_steps):
    loss += tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=Y[i], logits=Z_a[i]))
loss /= tf.cast(tf.size(X_raw), tf.float32)

train_step = tf.train.AdamOptimizer(0.01).minimize(loss)

sess.run(tf.global_variables_initializer())

In [47]:
def generate_repeating_dataset(n, n_x, n_steps):
    X = np.array([i % n_x for i in range(n)]).reshape(-1, n_steps)
    Y = np.array([(i+1) % n_x for i in range(n)]).reshape(-1, n_steps)
    return X, Y

n_batches = 3

inputs, outputs = generate_repeating_dataset(n_steps * n_batches, n_x, n_steps)

sess.run(loss, feed_dict = {
    X_raw: inputs, 
    Y_raw: outputs, 
    H_init: np.zeros((n_batches, n_h))})

#np.pad(, n_steps - (len(s) % n_steps), 'constant', constant_values='#').reshape(-1, n_steps)

1.0968565